# ICMP Pinger

Ping is a computer network application used to test whether a particular host is reachable across an IP
network. It is also used to self-test the network interface card of the computer or as a latency test. It works
by sending ICMP “echo reply” packets to the target host and listening for ICMP “echo reply” replies. The
"echo reply" is sometimes called a pong. Ping measures the round-trip time, records packet loss, and
prints a statistical summary of the echo reply packets received (the minimum, maximum, and the mean of
the round-trip times and in some versions the standard deviation of the mean).



Your task is to develop your own Ping application in Python. Your application will use ICMP but, in
order to keep it simple, will not exactly follow the official specification in RFC 1739. Note that you will
only need to write the client side of the program, as the functionality needed on the server side is built
into almost all operating systems.



You should complete the Ping application so that it sends ping requests to a specified host separated by
approximately one second. Each message contains a payload of data that includes a timestamp. After
sending each packet, the application waits up to one second to receive a reply. If one second goes by
without a reply from the server, then the client assumes that either the ping packet or the pong packet was
lost in the network (or that the server is down).

First, test your client by sending packets to localhost, that is, 127.0.0.1.
Then, you should see how your Pinger application communicates across the network by pinging servers
in different continents.

In [ ]:
from socket import *
import os
import struct
import time
import select
import sys

ICMP_ECHO_REQUEST = 8

def checksum(source):
    sum = 0
    countTo = (len(source) // 2) * 2
    count = 0

    while count < countTo:
        val = source[count + 1] * 256 + source[count]
        sum = sum + val
        sum = sum & 0xffffffff
        count += 2

    if countTo < len(source):
        sum += source[len(source) - 1]
        sum = sum & 0xffffffff

    sum = (sum >> 16) + (sum & 0xffff)
    sum += (sum >> 16)
    answer = ~sum & 0xffff
    answer = answer >> 8 | (answer << 8 & 0xff00)
    return answer

def sendOnePing(sock, destAddr, ID):
    header = struct.pack("bbHHh", ICMP_ECHO_REQUEST, 0, 0, ID, 1)
    data = struct.pack("d", time.time())
    packet = header + data
    chksum = checksum(packet)

    header = struct.pack("bbHHh", ICMP_ECHO_REQUEST, 0, htons(chksum), ID, 1)
    packet = header + data
    sock.sendto(packet, (destAddr, 1))

def receiveOnePing(sock, ID, timeout, destAddr):
    timeLeft = timeout
    while True:
        startSelect = time.time()
        ready = select.select([sock], [], [], timeLeft)
        timeInSelect = time.time() - startSelect

        if not ready[0]:
            return "Request timed out."

        timeReceived = time.time()
        recvPacket, addr = sock.recvfrom(1024)
        icmpHeader = recvPacket[20:28]
        type, code, recvChecksum, packetID, seq = struct.unpack("bbHHh", icmpHeader)

        if packetID == ID:
            timeSent = struct.unpack("d", recvPacket[28:28 + struct.calcsize("d")])[0]
            return f"Reply from {addr[0]}: time={(timeReceived - timeSent) * 1000:.2f}ms"

        timeLeft -= timeInSelect
        if timeLeft <= 0:
            return "Request timed out."

def doOnePing(destAddr, timeout):
    icmp = getprotobyname("icmp")
    try:
        sock = socket(AF_INET, SOCK_RAW, icmp)
    except PermissionError:
        print("Permission denied:")
        sys.exit(1)

    ID = os.getpid() & 0xFFFF
    sendOnePing(sock, destAddr, ID)
    delay = receiveOnePing(sock, ID, timeout, destAddr)
    sock.close()
    return delay

def ping(host, timeout=1):
    dest = gethostbyname(host)
    print(f"Pinging {host} [{dest}] with Python ICMP")
    try:
        while True:
            print(doOnePing(dest, timeout))
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nStopped.")

ping("google.com")
